Import Libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
from tensorflow import keras
from keras import backend as k
import time
from tensorflow.keras.callbacks import EarlyStopping

# 1. Process Audio into Spectogram

a function that returns audio in numeric representation

In [7]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    #wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

Read all audio files and sort

In [8]:
TRAIN = os.path.join('/content','drive','MyDrive','audio-data', 'Train')
TEST = os.path.join('/content','drive','MyDrive','audio-data', 'Test')
#TRAIN = os.path.join('audio-data', 'Train')
#TEST = os.path.join('audio-data', 'Test')
train = tf.data.Dataset.list_files(TRAIN+'/*.wav')
train = sorted(list(train.as_numpy_iterator()))
train = tf.data.Dataset.from_tensor_slices(train)
test = tf.data.Dataset.list_files(TEST+'/*.wav')
test = sorted(list(test.as_numpy_iterator()))
test = tf.data.Dataset.from_tensor_slices(test)

Add Labels

In [9]:
num_classes = 10
iterations = 0
i = 0
train_label = []
while iterations!=len(train):
    iterations +=1
    train_label.append(i)
    i += 1
    if i == num_classes :
        i = 0
train_label=keras.utils.to_categorical(train_label,num_classes)
trainings = tf.data.Dataset.zip((train, tf.data.Dataset.from_tensor_slices(train_label)))
#---------------------------------------------------------------#
iterations = 0
i = 0
test_label=[]
while iterations!=len(test):
    iterations +=1
    test_label.append(i)
    i += 1
    if i == num_classes :
        i = 0
test_label=keras.utils.to_categorical(test_label,num_classes)
testings = tf.data.Dataset.zip((test, tf.data.Dataset.from_tensor_slices(test_label)))

Build Preprocessing Function to get spectogram

In [10]:
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    #wav = wav[:48000]
    #zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    #wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

Convert all to Spectogram

In [11]:
# train data
x_train = trainings.map(preprocess)
x_train = x_train.cache()
x_train = x_train.shuffle(buffer_size=1000)
x_train = x_train.batch(16) # 16 at a time
x_train = x_train.prefetch(8)
# test data
x_test = testings.map(preprocess)
x_test = x_test.cache()
x_test = x_test.shuffle(buffer_size=1000)
x_test = x_test.batch(16) # 16 at a time
x_test = x_test.prefetch(8)

In [12]:
# test one batch
samples, labels = x_train.as_numpy_iterator().next()
print(samples.shape)
print('\n',labels[0:2],'\n...')

(16, 391, 257, 1)

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]] 
...


# 2. CNN no Attention

Design the CNN architecture

In [13]:
from keras.models import Sequential
from keras import layers

In [14]:
model=Sequential()
input_shape = (391, 257, 1)
model.add( layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape) )
model.add( layers.MaxPooling2D(pool_size=(2,2)) )
model.add( layers.Dropout(0.2) )
model.add( layers.Flatten() )
model.add( layers.Dense(32,activation='relu') )
model.add( layers.Dense(num_classes,activation='softmax') )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 389, 255, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 194, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 194, 127, 32)      0         
                                                                 
 flatten (Flatten)           (None, 788416)            0         
                                                                 
 dense (Dense)               (None, 32)                25229344  
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                        

Training

In [15]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy']
             )

In [16]:
early_stopping = EarlyStopping(monitor='accuracy', patience=3)
tic=time.time()
model.fit(x_train,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=x_test
          )
toc=time.time()
training_time=toc-tic

Epoch 1/15
75/75 [==============================] - 57s 560ms/step - loss: 2.1039 - accuracy: 0.5250 - val_loss: 0.9418 - val_accuracy: 0.7400
Epoch 2/15
75/75 [==============================] - 4s 55ms/step - loss: 0.4593 - accuracy: 0.8783 - val_loss: 0.7764 - val_accuracy: 0.7967
Epoch 3/15
75/75 [==============================] - 4s 53ms/step - loss: 0.1701 - accuracy: 0.9558 - val_loss: 0.9367 - val_accuracy: 0.7933
Epoch 4/15
75/75 [==============================] - 4s 54ms/step - loss: 0.0881 - accuracy: 0.9775 - val_loss: 1.3154 - val_accuracy: 0.8100
Epoch 5/15
75/75 [==============================] - 4s 55ms/step - loss: 0.2319 - accuracy: 0.9433 - val_loss: 1.8162 - val_accuracy: 0.6867
Epoch 6/15
75/75 [==============================] - 4s 52ms/step - loss: 0.1002 - accuracy: 0.9817 - val_loss: 1.5322 - val_accuracy: 0.7700
Epoch 7/15
75/75 [==============================] - 4s 54ms/step - loss: 0.0171 - accuracy: 0.9992 - val_loss: 1.7880 - val_accuracy: 0.7633
Epoch 8/15


In [17]:
tic=time.time()
test_loss, test_acc = model.evaluate(x_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

19/19 [==============================] - 0s 9ms/step - loss: 2.0850 - accuracy: 0.7700
Training Time = 95.6 s
Testing Time = 310.3 ms
Test Loss = 208.50 %:
Test Accuracy = 77.00 %:


# 3. CNN with Attention

Design the CNN architecture

In [18]:
inputs = layers.Input(shape=input_shape)

conv1 = layers.Conv2D(32,kernel_size=(3,3),activation='relu')(inputs)
pool1 = layers.MaxPool2D(pool_size=(2,2))(conv1)
#Attention1
attention_conv1 = layers.Conv2D(1, (3,3), padding='same', activation='sigmoid')(pool1)
attention_mul1 = layers.Multiply()([pool1, attention_conv1])
pool2 = layers.MaxPool2D(pool_size=(2,2))(attention_mul1)
##########
drop1 = layers.Dropout(0.2)(pool2)
flatten2 = layers.Flatten()(drop1)
dense2 = layers.Dense(32,activation='relu')(flatten2)
dense3 = layers.Dense(num_classes,activation='softmax')(dense2)

modelAtt = keras.Model(inputs=inputs, outputs=dense3)

modelAtt.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 391, 257, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 389, 255, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 194, 127, 32  0          ['conv2d_1[0][0]']               
                                )                                                             

Training

In [19]:
modelAtt.compile(optimizer=keras.optimizers.Adam(),
              loss= keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )

In [20]:
tic=time.time()
modelAtt.fit(x_train,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=x_test
          )
toc=time.time()
training_time=toc-tic

Epoch 1/15
75/75 [==============================] - 7s 63ms/step - loss: 1.4784 - accuracy: 0.5558 - val_loss: 0.9242 - val_accuracy: 0.7300
Epoch 2/15
75/75 [==============================] - 5s 64ms/step - loss: 0.5059 - accuracy: 0.8675 - val_loss: 0.8350 - val_accuracy: 0.7700
Epoch 3/15
75/75 [==============================] - 5s 65ms/step - loss: 0.2713 - accuracy: 0.9375 - val_loss: 0.8104 - val_accuracy: 0.7467
Epoch 4/15
75/75 [==============================] - 5s 63ms/step - loss: 0.1723 - accuracy: 0.9633 - val_loss: 1.5070 - val_accuracy: 0.7867
Epoch 5/15
75/75 [==============================] - 5s 60ms/step - loss: 0.3095 - accuracy: 0.9375 - val_loss: 1.4172 - val_accuracy: 0.7700
Epoch 6/15
75/75 [==============================] - 5s 63ms/step - loss: 0.1313 - accuracy: 0.9742 - val_loss: 1.6753 - val_accuracy: 0.7900
Epoch 7/15
75/75 [==============================] - 5s 62ms/step - loss: 0.1480 - accuracy: 0.9733 - val_loss: 1.4575 - val_accuracy: 0.7933
Epoch 8/15
75

In [21]:
tic=time.time()
test_loss, test_acc = modelAtt.evaluate(x_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

19/19 [==============================] - 0s 10ms/step - loss: 2.0579 - accuracy: 0.8133
Training Time = 69.2 s
Testing Time = 317.6 ms
Test Loss = 205.80 %:
Test Accuracy = 81.30 %:
